In [ ]:
import numpy as np
import pandas as pd

import ast
from ast import literal_eval

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report
from datetime import datetime

# ORGANISATION
- classification - alicia / maria 
- temporal evolution - anissa 
- majoritiy representation - virginie
- climate speaking proprotion - virginie 
- sentiment analysis - alicia / maria 
- readme data,set  - virginie
- readme methodes - alicia / maria 
- data story (site web) - anissa

- [doing] CLASSIFICATION - Based on the evolution of the profile of a person talking about climate in the media, can we predict what characteristics will a climate-involved person have in the future ? 
 
 - [doing] TEMPORAL EVOLUTION - Is there any temporal evolution in the mediatic coverage of the climate crisis ? If so, does it correlate with specific events ? 
 
 - [doing] MAJORITY REPRESENTATION - Who are the top 10 people that talks the most about climate change over the year ? Does it show an evolution in the general opinions made publicly avaiblable by mainstream media ? What is the current typical profile of a person talking about climate in the media ? Is it the same for other topic ? (Comparaison entre les caractéristiques de personnes les plus représentées dans le QuoteBank vs celles qui parlent du climat) (comparaison des nationalités avec une carte du monde) 
 
 - [x] CLIMATE SPEAKING PROPORTION - Are some communities excluded from the mediatic discussion surround climate change ? (Comparer la proportion de chacun des communautés dans QB en général avec la propotion des communautés dans les phrases qui parlent du climat) - plot climate vs non climate 
 
 - [x] NLP TOPIC CLASSIFICATION - What is the most represented climate crisis-related topic along the years ? Does it show an evolution in the general opinions made publicly avaiblable by mainstream media ? (Trouver les mots reliés au climat les plus représentés dans les quotes chaque années) 

- [x] SENTIMENT ANALYSIS - Vander Analysis among people that talks about climate

# A RUN POUR AVOIR SUBDATA (Year-month)

In [ ]:
dico_clean={} 
length = 0
for date in [2020, 2019, 2018, 2017, 2016, 2015]:
    dico_clean[date] = pd.read_csv(f'data/clean_quotes-{date}.bz2', compression='bz2')
    length += len(dico_clean[date]) #The length is used here to obtain the total number of quotes.

In [ ]:
subdata=pd.DataFrame()
for years, quotes in dico_clean.items() : 
    subdata = pd.concat([subdata, quotes])

In [ ]:
subdata.date.map(lambda y : datetime.strptime(y, '%Y-%m-%d %H:%M:%S'))
subdata.sort_values(by='date', inplace=True)
subdata['date'] = pd.to_datetime(subdata['date']).dt.strftime('%Y-%m')

# I- CLASSIFICATION

### 1.1 Logistic - baseline

In [ ]:
data_original = pd.read_csv("data/one_hot_data.bz2", compression = 'bz2')

In [ ]:
def split_set(data_to_split, ratio=0.8):
    mask = np.random.rand(len(data_to_split)) < ratio
    return [data_to_split[mask].reset_index(drop=True), data_to_split[~mask].reset_index(drop=True)]

In [ ]:
data_features = data_original.copy()
data_features.head(3)

In [ ]:
[train, test] = split_set(data_features)

In [ ]:
train_label=train.climate
train_features = train.drop('climate', axis=1)
print('Length of the train dataset : {}'.format(len(train)))

test_label=test.climate
test_features = test.drop('climate', axis=1)
print('Length of the test dataset : {}'.format(len(test)))

In [ ]:
means = train_features.mean()
stddevs = train_features.std()

train_features_std = pd.DataFrame()
for c in train_features.columns:
    train_features_std[c] = (train_features[c]-means[c])/stddevs[c]

# Use the mean and stddev of the training set
test_features_std = pd.DataFrame()
for c in test_features.columns:
    test_features_std[c] = (test_features[c]-means[c])/stddevs[c]

train_features_std.head()

In [ ]:
def compute_confusion_matrix(true_label, prediction_proba, decision_threshold=0.5): 
    
    predict_label = (prediction_proba[:,1]>decision_threshold).astype(int)   
                                                                                                                       
    TP = np.sum(np.logical_and(predict_label==1, true_label==1))
    TN = np.sum(np.logical_and(predict_label==0, true_label==0))
    FP = np.sum(np.logical_and(predict_label==1, true_label==0))
    FN = np.sum(np.logical_and(predict_label==0, true_label==1))
    
    confusion_matrix = np.asarray([[TP, FP],
                                    [FN, TN]])
    return confusion_matrix

def plot_confusion_matrix(confusion_matrix):
    [[TP, FP],[FN, TN]] = confusion_matrix
    label = np.asarray([['TP {}'.format(TP), 'FP {}'.format(FP)],
                        ['FN {}'.format(FN), 'TN {}'.format(TN)]])
    
    df_cm = pd.DataFrame(confusion_matrix, index=['Yes', 'No'], columns=['Positive', 'Negative']) 
    
    return sns.heatmap(df_cm, cmap='YlOrRd', annot=label, annot_kws={"size": 16}, cbar=False, fmt='')

def compute_all_score(confusion_matrix, t=0.5):
    [[TP, FP],[FN, TN]] = confusion_matrix.astype(float)
    
    accuracy =  (TP+TN)/np.sum(confusion_matrix)
    
    precision_positive = TP/(TP+FP) if (TP+FP) !=0 else np.nan
    precision_negative = TN/(TN+FN) if (TN+FN) !=0 else np.nan
    
    recall_positive = TP/(TP+FN) if (TP+FN) !=0 else np.nan
    recall_negative = TN/(TN+FP) if (TN+FP) !=0 else np.nan

    F1_score_positive = 2 *(precision_positive*recall_positive)/(precision_positive+recall_positive) if (precision_positive+recall_positive) !=0 else np.nan
    F1_score_negative = 2 *(precision_negative*recall_negative)/(precision_negative+recall_negative) if (precision_negative+recall_negative) !=0 else np.nan

    return [t, accuracy, precision_positive, recall_positive, F1_score_positive, precision_negative, recall_negative, F1_score_negative]

In [ ]:
logistic = LogisticRegression(solver='lbfgs', max_iter=10000)
logistic.fit(train_features_std,train_label)

In [ ]:
prediction_proba = logistic.predict_proba(test_features_std)

In [ ]:
confusion_matrix_05 = compute_confusion_matrix(test_label, prediction_proba, 0.5 )
plt.figure(figsize = (4,3)) 
ax = plot_confusion_matrix(confusion_matrix_05)
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.title('Confusion matrix for a 0.5 threshold')

In [ ]:
[t, accuracy, precision_positive, recall_positive, F1_score_positive, \
    precision_negative, recall_negative, F1_score_negative] = compute_all_score(confusion_matrix_05)

print("The accuracy of this model is {0:1.3f}".format(accuracy))
print("For the positive case, the precision is {0:1.3f}, the recall is {1:1.3f} and the F1 score is {2:1.3f}"\
      .format(precision_positive, recall_positive, F1_score_positive))
print("For the negative case, the precision is {0:1.3f}, the recall is {1:1.3f} and the F1 score is {2:1.3f}"\
      .format(precision_negative, recall_negative, F1_score_negative))

In [ ]:
tmp = []
for name, value in zip(train_features_std.columns, logistic.coef_[0]):
    tmp.append({"name": name, "value": value})
    
features_coef = pd.DataFrame(tmp).sort_values("value")
features_coef.head()

In [ ]:
plt.subplots(figsize=(6,40))
plt.barh(features_coef.name, features_coef.value, alpha=1)

### 1.2 Logistic - standardized vs not

### 1.2 Logistic - remove features vs not 

### 2.1 Random forest 

### 3.1 Gradient Boosting

In [ ]:
gradient_booster = GradientBoostingClassifier(learning_rate=0.1)
gradient_booster.fit(train_features,train_label)
print(classification_report(test_label,gradient_booster.predict(test_feature)))

In [ ]:
print(gradient_booster.feature_importances_)

# plot
pyplot.bar(range(len(gradient_booster.feature_importances_)), gradient_booster.feature_importances_)
pyplot.show()

# Temporal Evolution

> Years and month 

In [ ]:
ax=sns.catplot(data=subdata, x = 'date',  kind="count", 
               palette="ch:.25",height=7, aspect=2)
ax.set_xticklabels(rotation=80)

ax.set(xlabel='Time', ylabel='Number of quotes', 
       title= 'frequency of climate related quotes over the time ', yscale='log') #We chose a log scale for better data visulalisation. 

sns.despine()

> Years

In [ ]:
subdata_years = subdata.copy()
subdata_years['date'] = pd.to_datetime(subdata_years['date']).dt.strftime('%Y')


ax=sns.catplot(data=subdata_years, x = 'date',  kind="count", 
               palette="ch:.25",height=4, aspect=1.8)

ax.set_xticklabels(rotation=80)

ax.set(xlabel='Time', ylabel='Number of quotes', 
       title= 'frequency of climate related quotes over the years ', yscale='log') #We chose a log scale for better data visulalisation. 

sns.despine()

# Majority representation

In [ ]:
top_10 = subdata.copy()
speaker_time = top_10[['speaker','date','numOccurrences']]
count_top_10 = speaker_time.groupby('speaker')['numOccurrences'].agg(['count']).sort_values('count', ascending=False).head(10)

In [ ]:
count_top_10

In [ ]:
speaker_list = speaker_time['speaker'].tolist()

x = 0 
big_ls = []

for i in count_top_10.index:
    ls = []
    x = x+1
    if i in speaker_list:
        ls.append(speaker_time['date'])
        big_ls.append(ls)

count_top_10['date'] = big_ls
count_top_10

In [ ]:
x_speaker = count_top_10.index.tolist()
y_count = count_top_10['count']

count_top_10.plot.bar()

# Climate speaking proprotion

# NLP 